# Sprint 17: Telecom Churn Forecasting Project

# Overview

The goal of the project is to forecast customer churn for the telecom operator Interconnect. If a user is predicted to churn, they will be targeted with retention offers. The project will involve building a model to predict churn, evaluating it using the AUC-ROC metric, and optimizing it to maximize accuracy and AUC-ROC.

---

## Data Exploration and Preprocessing

### Data Loading
Let's start by loading the datasets from `/datasets/final_provider/`.

In [2]:
import pandas as pd

In [4]:
# File paths for datasets
contract_path = '/datasets/final_provider/contract.csv'
personal_path = '/datasets/final_provider/personal.csv'
internet_path = '/datasets/final_provider/internet.csv'
phone_path = '/datasets/final_provider/phone.csv'

# Load datasets
contract = pd.read_csv(contract_path)
personal = pd.read_csv(personal_path)
internet = pd.read_csv(internet_path)
phone = pd.read_csv(phone_path)

# Display the first few rows of each dataset
display(contract.head(), personal.head(), internet.head(), phone.head())

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65


,customerID,gender,SeniorCitizen,Partner,Dependents
0,7590-VHVEG,Female,0,Yes,No
1,5575-GNVDE,Male,0,No,No
2,3668-QPYBK,Male,0,No,No
3,7795-CFOCW,Male,0,No,No
4,9237-HQITU,Female,0,No,No


,customerID,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,7590-VHVEG,DSL,No,Yes,No,No,No,No
1,5575-GNVDE,DSL,Yes,No,Yes,No,No,No
2,3668-QPYBK,DSL,Yes,Yes,No,No,No,No
3,7795-CFOCW,DSL,Yes,No,Yes,Yes,No,No
4,9237-HQITU,Fiber optic,No,No,No,No,No,No


,customerID,MultipleLines
0,5575-GNVDE,No
1,3668-QPYBK,No
2,9237-HQITU,No
3,9305-CDSKC,Yes
4,1452-KIOVK,Yes


### Data Overview

In [7]:
# Checking data structure
print("Contract Data Info:")
contract.info()
print()
print("\nPersonal Data Info:")
personal.info()
print()
print("\nInternet Data Info:")
internet.info()
print()
print("\nPhone Data Info:")
phone.info()
print()

# Checking for duplicates
print(f"Contract duplicates: {contract.duplicated().sum()}")
print(f"Personal duplicates: {personal.duplicated().sum()}")
print(f"Internet duplicates: {internet.duplicated().sum()}")
print(f"Phone duplicates: {phone.duplicated().sum()}")


Contract Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
dtypes: float64(1), object(7)
memory usage: 440.3+ KB


Personal Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     7043 non-null   object
 1   gender         7043 non-null   object
 2   SeniorCitizen  7043 non-null   int64 
 3   Pa

### Data Cleaning

In [8]:
# Checking for missing values
print("Missing values in contract data:")
print(contract.isna().sum())

print("Missing values in personal data:")
print(personal.isna().sum())

print("Missing values in internet data:")
print(internet.isna().sum())

print("Missing values in phone data:")
print(phone.isna().sum())

# Fill missing values with appropriate strategies
internet.fillna('No', inplace=True)
phone.fillna('No', inplace=True)
# If other strategies are necessary, apply here

# Merge datasets on 'customerID'
merged_data = contract.merge(personal, on='customerID', how='left')\
                      .merge(internet, on='customerID', how='left')\
                      .merge(phone, on='customerID', how='left')

# Display the merged dataset
display(merged_data.head())


Missing values in contract data:
customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
dtype: int64
Missing values in personal data:
customerID       0
gender           0
SeniorCitizen    0
Partner          0
Dependents       0
dtype: int64
Missing values in internet data:
customerID          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
dtype: int64
Missing values in phone data:
customerID       0
MultipleLines    0
dtype: int64


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85,Female,0,Yes,No,DSL,No,Yes,No,No,No,No,NaN
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No
3,7795-CFOCW,2016-05-01,No,One year,No,Bank transfer (automatic),42.30,1840.75,Male,0,No,No,DSL,Yes,No,Yes,Yes,No,No,NaN
4,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No


### Data Inspection

In [11]:
# Inspect the merged data
merged_data.info()

# Check for any remaining missing values
print("Missing values in merged data:")
print(merged_data.isna().sum())

# Descriptive statistics
print("Descriptive statistics for numeric features:")
display(merged_data.describe())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   BeginDate         7043 non-null   object 
 2   EndDate           7043 non-null   object 
 3   Type              7043 non-null   object 
 4   PaperlessBilling  7043 non-null   object 
 5   PaymentMethod     7043 non-null   object 
 6   MonthlyCharges    7043 non-null   float64
 7   TotalCharges      7043 non-null   object 
 8   gender            7043 non-null   object 
 9   SeniorCitizen     7043 non-null   int64  
 10  Partner           7043 non-null   object 
 11  Dependents        7043 non-null   object 
 12  InternetService   5517 non-null   object 
 13  OnlineSecurity    5517 non-null   object 
 14  OnlineBackup      5517 non-null   object 
 15  DeviceProtection  5517 non-null   object 
 16  TechSupport       5517 non-null   object 


,MonthlyCharges,SeniorCitizen
count,7043.000000,7043.000000
mean,64.761692,0.162147
std,30.090047,0.368612
min,18.250000,0.000000
25%,35.500000,0.000000
50%,70.350000,0.000000
75%,89.850000,0.000000
max,118.750000,1.000000


# Proposed Work Plan for Telecom Churn Forecasting Project

## Data Exploration and Preprocessing
- **Load and inspect the data**: Load all relevant datasets (contract, personal, internet, and phone) and understand their structure (data types, missing values, duplicates).
- **Clean the data**: Address missing values by filling in where appropriate (e.g., assuming 'No' for missing service data) and removing duplicates.
- **Merge the datasets**: Combine all the datasets into a single DataFrame using customerID as the key.

## Feature Engineering
- **Create new features**: Generate relevant features like the total number of services subscribed by the customer, contract duration, monthly charges, and tenure.
- **Convert categorical data**: Use one-hot encoding or label encoding to convert categorical features into numerical form.

## Data Splitting
- **Define target variable**: Create a target variable that identifies customer churn (i.e., EndDate != 'No').
- **Split data**: Use the `train_test_split()` function to divide the data into training and test sets.

## Model Training
- **Baseline Model**: Train a Logistic Regression model to establish a baseline performance using the AUC-ROC and accuracy metrics.
- **Advanced Models**: Train additional models like Random Forest, XGBoost, and SVM. Use hyperparameter tuning techniques (e.g., GridSearchCV) to optimize model performance.

## Model Evaluation
- **Evaluate the models**: Use the AUC-ROC score as the primary evaluation metric. Also, calculate the accuracy of the models on the test set.
- **Compare the results**: Identify the model with the highest AUC-ROC score and assess whether it meets the performance criteria (AUC-ROC ≥ 0.88).

## Final Model Deployment
- **Deploy the best model**: Once the best-performing model is identified, evaluate its final performance on the test set and document the results.
- **Report findings**: Provide a summary of the data insights and model performance, along with recommendations for using the model in churn prevention campaigns.

## Clarifying Questions for Stakeholders
- What is the expected churn rate threshold for which interventions should be applied?
- Should the model prioritize minimizing false negatives (i.e., failing to detect churn)?
- Are there any specific business constraints for handling missing data in the `paymentMethod` or service fields?
- Can customers suspend their contracts temporarily, and how should we handle these cases in our analysis?
